In [12]:
import FinanceDataReader as fdr
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup

# 주식 산업별 거래량 데이터 모으기 

In [2]:
url = "https://finance.naver.com/sise/sise_group.naver?type=upjong"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
soup = soup.find(class_="type_1").findAll('a')

page_num = [] # for finding stock name
industry_name = []
for i in range(len(soup)):
    if soup[i].string != "기타" and soup[i].string !="창업투자": # not need '기타' category because that have futures(선물) 
        page_num.append(soup[i].get("href"))
        industry_name.append(soup[i].string)
        
classfied_stock = {}
whole_stock_name = []
for i in range(len(page_num)):
    res = requests.get("https://finance.naver.com/" + page_num[i])
    soup = BeautifulSoup(res.text,'lxml')
    soup = soup.find('tbody').find_all('a')
    for j in range(len(soup)):
        stock_name = soup[j].getText()
        if stock_name != "":
            classfied_stock[stock_name] = industry_name[i]
            whole_stock_name.append(stock_name)
            
            
classfied_stock = pd.DataFrame(list(classfied_stock.items()),columns=['Stock_Name', 'Sectors'])
stock_code = pd.read_csv("stock_code.csv",encoding='CP949')
stock_code = stock_code[['단축코드','한글 종목약명']]
stock_code = stock_code[stock_code['한글 종목약명'].isin(whole_stock_name)]
stock_code = stock_code.rename(columns = {'한글 종목약명':'Stock_Name'})

classified_stock = classfied_stock.merge(stock_code,on='Stock_Name')

In [3]:
Sector_SC = None
Sector_Bio = None
Sector_Car = None
Sector_Gas = None
Sector_Game = None

In [4]:
target_sector = [Sector_SC,Sector_Bio,Sector_Car,Sector_Gas,Sector_Game]
sectors = ["반도체와반도체장비" , "제약" , "자동차" , "석유와가스" , "게임엔터테인먼트"]
stock_number = []
start_time = pd.Timestamp(year=2021, month=1, day=1)


for i in range(5):
    stock_code = classified_stock[classified_stock['Sectors'] == sectors[i]]['단축코드']
    stock_number.append(len(stock_code))
    for code in stock_code.values:
        stock_price = None
        stock_price = fdr.DataReader(code,start = "2021-11-09",exchange="KRX")
        stock_price['num_unlisted'] = 0
        stock_price.reset_index(inplace=True)
        min = stock_price['Date'].min()
        
        if code == stock_code.values[0]:
            target_sector[i] = stock_price
        else:
            if(len(target_sector[i])!=len(stock_price)):
                target_sector[i].loc[(target_sector[i]['Date']<min),'num_unlisted'] = target_sector[i].loc[(target_sector[i]['Date']<min),'num_unlisted'].values + 1
                target_sector[i].loc[(target_sector[i]['Date']>=min),'Volume'] = target_sector[i].loc[(target_sector[i]['Date']>=min),'Volume'].values+stock_price['Volume'].values
            else:
                target_sector[i]['Volume'] = target_sector[i]['Volume'] + stock_price['Volume']

In [5]:
result = None
for i in range(5):
    target_sector[i]['Volume'] = target_sector[i]['Volume']/(stock_number[i] - target_sector[i]['num_unlisted'])
    target_sector[i].rename(columns = {'Volume' : sectors[i]}, inplace = True)
    if i==0:
        result = target_sector[i]
    else:
        result = pd.merge(result,target_sector[i][[sectors[i]]+["Date"]],on="Date") 
        
result = result[["Date"]+sectors]

In [6]:
result

,Date,반도체와반도체장비,제약,자동차,석유와가스,게임엔터테인먼트
0,2021-11-09,5.649465e+05,211612.152318,334540.000,4.199246e+06,1.519503e+06
1,2021-11-10,6.160860e+05,328178.688742,633348.875,3.205902e+06,3.367234e+06
2,2021-11-11,6.205119e+05,236115.913907,386556.000,1.870801e+06,3.351900e+06
3,2021-11-12,7.300724e+05,268352.894040,349315.375,1.382111e+06,1.540664e+06
4,2021-11-15,1.980573e+06,374094.258278,306255.000,2.468123e+06,1.155049e+06
5,2021-11-16,1.125604e+06,274601.119205,415799.125,1.240691e+06,1.680001e+06
6,2021-11-17,1.007052e+06,230441.688742,411796.250,2.036822e+06,2.607056e+06
7,2021-11-18,7.054886e+05,213678.026490,343106.125,2.270077e+06,3.997607e+06
8,2021-11-19,2.673618e+06,184157.231788,404964.250,1.163741e+06,1.407674e+06
9,2021-11-22,2.299030e+06,180847.682119,680434.250,1.156311e+06,1.519572e+06


# ETF 거래량 모으기

In [29]:
US_SC_ETF = ['SOXX' ,'SMH' ,'SOXL']
US_BIO_ETF = ['ARKG', 'XLV', 'IBB']
US_CAR_ETF = ['CARZ', 'DRIV', 'ARKK']
US_GAS_ETF = ['USO','BOIL','XLE']
US_GM_ETF = ["ESPO" ,'HERO' ,'GAMR']

SC_ETF = None
BIO_ETF = None
CAR_ETF = None
GAS_ETF = None
GM_ETF = None
etf_price = None

Total_ETF = [US_SC_ETF,US_BIO_ETF,US_CAR_ETF,US_GAS_ETF,US_GM_ETF]
DF_ETF = [SC_ETF,BIO_ETF,CAR_ETF,GAS_ETF,GM_ETF]


In [30]:
for i in range(5):
    for etf_name in Total_ETF[i]:
        etf_price =fdr.DataReader(symbol = etf_name,start = "2021-11-09")
        #etf_price.drop(columns={"High","Low","Change"},inplace =True)
        if etf_name == Total_ETF[i][0]:
            DF_ETF[i] = etf_price
        else:
            DF_ETF[i] = DF_ETF[i] + etf_price
            
Natural_Gas = fdr.DataReader('NG',start = "2021-11-09")
USO = fdr.DataReader("USO",start = "2021-11-09")
DF_ETF[3]['NG_Volume'] = Natural_Gas['Volume']
DF_ETF[3]['USO_Volume'] = USO['Volume'] 
currency = fdr.DataReader("USD/KRW",start = "2021-11-09")
currency2 = fdr.DataReader("DX",start = "2021-11-09")

In [31]:
#etf_result = result
etf_result = DF_ETF[0]

etf_result["SC_ETF"] = DF_ETF[0]['Volume']
etf_result["BIO_ETF"] = DF_ETF[1]['Volume']
etf_result["CAR_ETF"] = DF_ETF[2]['Volume']
etf_result["GAS_ETF"] = DF_ETF[3]['Volume']
etf_result["NG_VOLUME"] = DF_ETF[3]['NG_Volume']
etf_result["USO_VOLUME"] = DF_ETF[3]['USO_Volume']
etf_result["GM_ETF"] = DF_ETF[4]['Volume']
etf_result['Exchange'] = currency['Close']
etf_result['DolloarIndex'] = currency2['Close']
etf_result = etf_result.drop(columns={"Close","Open","Volume"})


etf_result.reset_index(inplace=True)

In [32]:
ETF_Result = pd.merge(etf_result,result,on="Date")

In [43]:
ETF_Results = ETF_Result[ETF_Result.columns.difference(ETF_Result.columns[1:4])]

In [48]:
ETF_Results.to_csv("NewETF_Result.csv")

In [2]:
import FinanceDataReader as fdr
import pandas as pd

NYSE = fdr.StockListing("NYSE")

100%|██████████| 3409/3409 [00:03<00:00, 867.24it/s] 


In [7]:
US_Sector_SC = None
US_Sector_Bio = None
US_Sector_Car = None
US_Sector_Gas = None
US_Sector_Game = None

target_sector = [US_Sector_SC,US_Sector_Bio,US_Sector_Car,US_Sector_Gas,US_Sector_Game]

In [13]:
#pip install -U finance-datareader --> 터미널에서 설치 필요해요 

import FinanceDataReader as fdr
import pandas as pd
import FinanceDataReader as fdr
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup

NYSE = fdr.StockListing("NYSE")

SC_Stock = NYSE[NYSE['Industry']=="반도체 및 반도체 장비"]['Symbol'].values
BIO_Stock = NYSE[NYSE['Industry']=="제약"]['Symbol'].values
GAS_Stock = NYSE[NYSE['Industry']=="오일 및 가스"]['Symbol'].values
CAR_Stock = NYSE[NYSE['Industry']=="자동차 및 자동차 부품"]['Symbol'].values
GM_Stock = NYSE[NYSE['Industry']=="소프트웨어 및 IT서비스"]['Symbol'].values
GM_Stock = GM_Stock[:-1]
BIO_Stock = BIO_Stock[:-3]
CAR_Stock = list(CAR_Stock)
CAR_Stock.remove('APTV PR A')

gas_remove = ['XEC','GLP PR B','NGL PR B','GLP PR A','NGL PR C','RDS.A','PBR.A','RDS.B','COG','CIVI','EQT']
GAS_Stock = list(GAS_Stock)
for i in gas_remove:
    GAS_Stock.remove(i)
    
gm_remove = ['SOGO','QVCC','QVCD']
GM_Stock = list(GM_Stock)
for i in gm_remove:
    GM_Stock.remove(i)
    
Total_Stock = [SC_Stock]+[BIO_Stock]+[CAR_Stock]+[GAS_Stock]+[GM_Stock]

US_Sector_Gas = None
US_Sector_Game = None
A = [None,None,None,US_Sector_Gas,US_Sector_Game]
for i in range(3,5):
    symbols = Total_Stock[i]
    for symbol in symbols:
        stock_price = None
        stock_price = fdr.DataReader(symbol,start = "2021-11-08",end = "2021-12-06")
        stock_price['num_unlisted'] = 0
        stock_price.reset_index(inplace=True)
        
        if symbol == symbols[0]:
            A[i] = stock_price
            A[i].rename(columns ={'Volume':'Total_Volume'},inplace=True)
        else:
            A[i] = pd.merge(stock_price,A[i][['Date','Total_Volume']],on='Date',how='outer')
            A[i].fillna(0,inplace=True)
            
            if(len(A[i])!=len(stock_price)):
                A[i].loc[A[i]['Volume']==0,'num_unlisted'] = A[i].loc[A[i]['Volume']==0,'num_unlisted'].values + 1
                A[i]['Total_Volume'] = A[i]['Total_Volume'] + A[i]['Volume']
            else:
                A[i]['Total_Volume'] = A[i]['Total_Volume'] + A[i]['Volume']

100%|██████████| 3411/3411 [00:11<00:00, 294.59it/s]


KeyError: 'Date'

In [16]:
A[3]['']

,Date,Close,Open,High,Low,Volume,Change,num_unlisted,Total_Volume
0,2021-11-08,6.070,6.150,6.200,5.955,5040.0,-0.0114,0.0,251448720.0
1,2021-11-09,6.400,6.094,6.450,6.050,14140.0,0.0544,0.0,247307500.0
2,2021-11-10,6.190,6.290,6.300,6.190,4820.0,-0.0328,0.0,275891450.0
3,2021-11-11,6.100,6.250,6.250,6.010,8620.0,-0.0145,0.0,228409530.0
4,2021-11-12,6.400,6.410,6.600,6.254,7690.0,0.0492,0.0,210005230.0
5,2021-11-15,6.360,6.600,6.600,6.345,5680.0,-0.0062,0.0,228020490.0
6,2021-11-16,5.750,6.300,6.300,5.480,54780.0,-0.0959,0.0,220371770.0
7,2021-11-17,5.810,5.650,5.810,5.620,5360.0,0.0104,0.0,249188170.0
8,2021-11-18,5.820,5.855,5.881,5.780,3810.0,0.0017,0.0,240259230.0
9,2021-11-19,5.760,5.870,5.870,5.557,4670.0,-0.0103,0.0,327845130.0


In [ ]:
example = pd.read_csv("New3Sectors.csv")

In [122]:
symbols = Total_Stock[4]
for symbol in symbols:
        stock_price = None
        stock_price = fdr.DataReader(symbol,start = "2021-11-08")
        stock_price['num_unlisted'] = 0
        stock_price.reset_index(inplace=True)
        
        A[4] = pd.merge(stock_price,A[4][['Date','Total_Volume']],on='Date',how='outer')
        A[4].fillna(0,inplace=True)
        if symbol == symbols[0]:
            A[4] = stock_price
            A[4].rename(columns ={'Volume':'Total_Volume'},inplace=True) 
        else:        
            if(len(A[4])!=len(stock_price)):
                A[4].loc[A[4]['Volume']==0,'num_unlisted'] = A[4].loc[A[4]['Volume']==0,'num_unlisted'].values + 1
                A[4]['Total_Volume'] = A[4]['Total_Volume'] + A[4]['Volume']
            else:
                A[4]['Total_Volume'] = A[4]['Total_Volume'] + A[4]['Volume']   

KeyError: 'Date'

In [123]:
New3day_result

,Unnamed: 0,Date,US_SC_SECTOR_VOLUME,US_BIO_SECTOR_VOLUME,US_CAR_SECTOR_VOLUME,US_GAS_SECTOR_VOLUME
0,0,2021-11-08,2.535311e+06,5441681.6,7.541805e+06,2.733588e+06
1,1,2021-11-09,1.855284e+06,3733780.0,7.601800e+06,2.688693e+06
2,2,2021-11-10,1.832486e+06,4625422.0,9.469705e+06,2.999048e+06
3,3,2021-11-11,1.421550e+06,4023852.4,6.460999e+06,2.482884e+06
4,4,2021-11-12,1.259206e+06,4014367.6,5.390027e+06,2.282784e+06
5,5,2021-11-15,1.142189e+06,3685970.8,5.502127e+06,2.478639e+06
6,6,2021-11-16,1.514936e+06,4017942.0,5.356496e+06,2.395713e+06
7,7,2021-11-17,1.525636e+06,4149557.6,5.725131e+06,2.708906e+06
8,8,2021-11-18,1.871695e+06,4029785.6,6.527017e+06,2.611856e+06
9,9,2021-11-19,1.543554e+06,5190828.4,4.515432e+06,3.563693e+06


In [116]:
GM_Stock.index(symbol)

72

In [90]:
New3day_result.to_csv("sector5excluded.csv")
sectors = ["US_SC_SECTOR_VOLUME" ,"US_BIO_SECTOR_VOLUME","US_CAR_SECTOR_VOLUME","US_GAS_SECTOR_VOLUME", "US_GM_SECTOR_VOLUME"]
New3day_result[sectors[4]] = A[4][sectors[4]]

,Unnamed: 0,Date,US_SC_SECTOR_VOLUME,US_BIO_SECTOR_VOLUME,US_CAR_SECTOR_VOLUME
0,0,2021-11-08,2.535311e+06,5441681.6,7.541805e+06
1,1,2021-11-09,1.855284e+06,3733780.0,7.601800e+06
2,2,2021-11-10,1.832486e+06,4625422.0,9.469705e+06
3,3,2021-11-11,1.421550e+06,4023852.4,6.460999e+06
4,4,2021-11-12,1.259206e+06,4014367.6,5.390027e+06
5,5,2021-11-15,1.142189e+06,3685970.8,5.502127e+06
6,6,2021-11-16,1.514936e+06,4017942.0,5.356496e+06
7,7,2021-11-17,1.525636e+06,4149557.6,5.725131e+06
8,8,2021-11-18,1.871695e+06,4029785.6,6.527017e+06
9,9,2021-11-19,1.543554e+06,5190828.4,4.515432e+06


In [17]:
stock_number = []
for i in Total_Stock:
    stock_number.append(len(i))

In [18]:
sectors = ["US_SC_SECTOR_VOLUME" ,"US_BIO_SECTOR_VOLUME","US_CAR_SECTOR_VOLUME","US_GAS_SECTOR_VOLUME", "US_GM_SECTOR_VOLUME"]

# US_result = None

A[3]['Total_Volume'] = A[3]['Total_Volume']/(stock_number[3] - A[3]['num_unlisted'])
A[3].rename(columns = {'Total_Volume' : sectors[3]}, inplace = True)
#     if i==0:
#         US_result = A[i]
#     else:
#         US_result = pd.merge(US_result,A[i][[sectors[i]]+["Date"]],on="Date") 
        
#US_result = US_result[["Date"]+sectors]

In [51]:
US_result.to_csv("New3Sectors.csv")

In [ ]:
New3day_result = pd.read_csv("New3Sectors.csv")
ETF_Results = pd.read_csv("NewETF_Result.csv")
New_Result = pd.read_csv("Us_Added_Stock_Info.csv")

In [156]:
#Total_Result = pd.merge(US_result,result,on='Date')
Total_Result = pd.merge(ETF_Results,Total_Result,on='Date')

In [166]:
add = pd.read_csv("BondRate.csv")
add['Date'] = pd.to_datetime(add['Date'])
Total_Result = pd.merge(add,Total_Result,on='Date')

In [159]:
Total_Result.sort_values(by =['Date'],ascending=False,inplace = True)
Total_Result.reset_index(drop=True,inplace=True)

In [162]:
NanColumns = ['DolloarIndex','NG_VOLUME']
for name in NanColumns:
    NanList  = Total_Result.loc[Total_Result[name].isna()].index
    for i in NanList:
        if i!=0:
            Total_Result.loc[i,name] = (Total_Result.loc[i+1,name] +Total_Result.loc[i-1,name])/2
        else:
            Total_Result.loc[i,name] = Total_Result.loc[i+1,name]

In [170]:
before_change = list(Total_Result.columns[-5:].values)

In [174]:
for i in range(5):
    Total_Result.rename(columns={before_change[i]:sectors[i]},inplace =True)

In [182]:
Total_Result.drop(columns=Total_Result.columns[11:16],inplace=True)

In [184]:
Total_Result.to_csv("Us_Added_Stock_Info.csv")

In [3]:
Total_Result= pd.read_csv("Us_Added_Stock_Info.csv")

In [4]:
Total_Result.drop(columns="Unnamed: 0", inplace= True)

In [5]:
sectors = ["US_SC_SECTOR_VOLUME" ,"US_BIO_SECTOR_VOLUME","US_CAR_SECTOR_VOLUME" , "US_GAS_SECTOR_VOLUME", "US_GM_SECTOR_VOLUME"]
Total_Result[['Date']+sectors]

,Date,US_SC_SECTOR_VOLUME,US_BIO_SECTOR_VOLUME,US_CAR_SECTOR_VOLUME,US_GAS_SECTOR_VOLUME,US_GM_SECTOR_VOLUME
0,2021-01-04,1.943012e+06,4751732.4,1.293087e+07,3.538662e+06,2.507433e+06
1,2021-01-05,1.567917e+06,4572254.8,9.395011e+06,5.654613e+06,2.165651e+06
2,2021-01-06,1.774856e+06,4902517.2,1.111446e+07,5.803685e+06,2.303577e+06
3,2021-01-07,1.901702e+06,3893866.8,8.299572e+06,3.772788e+06,2.439438e+06
4,2021-01-08,2.392240e+06,4111595.6,1.032720e+07,3.450589e+06,2.391136e+06
...,...,...,...,...,...,...
206,2021-11-11,1.421550e+06,4023852.4,6.460999e+06,2.535479e+06,4.158503e+06
207,2021-11-12,1.259206e+06,4014367.6,5.390027e+06,2.310701e+06,3.290843e+06
208,2021-11-15,1.142189e+06,3685970.8,5.502127e+06,2.530865e+06,2.573064e+06
209,2021-11-16,1.514936e+06,4017942.0,5.356496e+06,2.428729e+06,3.093370e+06


In [6]:
Total_Result.columns[:-5]

Index(['Date', 'KBondRate', 'SC_ETF', 'BIO_ETF', 'CAR_ETF', 'GAS_ETF',
       'NG_VOLUME', 'USO_VOLUME', 'GM_ETF', 'Exchange', 'DolloarIndex',
       'US_SC_SECTOR_VOLUME', 'US_BIO_SECTOR_VOLUME', 'US_CAR_SECTOR_VOLUME',
       'US_GAS_SECTOR_VOLUME', 'US_GM_SECTOR_VOLUME'],
      dtype='object')

In [7]:
sectors = ["반도체와반도체장비" , "제약" , "자동차" , "석유와가스" , "게임엔터테인먼트"]
Total_Result[['Date']+sectors]

,Date,반도체와반도체장비,제약,자동차,석유와가스,게임엔터테인먼트
0,2021-01-04,2.107285e+06,8.335759e+05,1822792.375,2.939573e+06,4.667284e+05
1,2021-01-05,1.715251e+06,1.203097e+06,1187411.625,3.210694e+06,4.302828e+05
2,2021-01-06,1.824799e+06,2.796216e+06,983732.000,5.121786e+06,5.988308e+05
3,2021-01-07,2.008111e+06,1.208024e+06,1104469.625,2.359454e+06,5.879534e+05
4,2021-01-08,2.171529e+06,9.520995e+05,6781590.875,2.176021e+06,7.291916e+05
...,...,...,...,...,...,...
206,2021-11-11,6.205119e+05,2.361159e+05,386556.000,1.870801e+06,3.351900e+06
207,2021-11-12,7.300724e+05,2.683529e+05,349315.375,1.382111e+06,1.540664e+06
208,2021-11-15,1.980573e+06,3.740943e+05,306255.000,2.468123e+06,1.155049e+06
209,2021-11-16,1.125604e+06,2.746011e+05,415799.125,1.240691e+06,1.680001e+06


In [19]:
ETF_Result.to_csv("2020Data.csv")